# JWST NIRSpec Observation Visualization Tool (NOVT): Interactive Tools
-----------------------------------------------------------------------
**Author**: Melanie Clarke (mclarke@stsci.edu) | **Latest update**: May 18, 2023.
<a class="anchor" id="title"></a>

## Notebook Goals
Use interactive NOVT tools to determine NIRSpec and NIRCam apertures and
visibility for a specific target within a range of dates.


## Table of contents
1. [Introduction](#intro)
2. [Read in data](#data)
3. [Determine visibility ranges](#visibility)
4. [Set aperture locations](#apertures)
5. [Check target coverage](#targets)
6. [Save DS9 regions](#regions)

<a class="anchor" id="intro"></a>
## Introduction

The NOVT package contains tools intended to assist in visualizing and planning NIRSpec observations, especially for multi-object spectroscopy (MOS) observations that require NIRCam pre-imaging observations.

STScI plans to host a web application that uses NOVT tools to plot target visibility and display NIRSpec and NIRCam apertures over a FITS image.  This notebook demonstrates how to use the same tools in a local Jupyter server, allowing you to use the application without uploading your data across the internet.

To start, we import some necessary packages.

In [ ]:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.wcs import WCS
from jdaviz import Imviz

from jwst_novt.interact import (
    ControlInstruments,
    ShowOverlays,
    ShowTimeline,
    UploadData,
    SaveOverlays,
)

<a class="anchor" id="data"></a>
## Read in data
For this example, we will use a FITS image of M51 (NGC 5194 and NGC 5195) and an associated catalog of sources. The following commands will download the data and cache it locally for you to use.

In [ ]:
image_link = (
    "https://archive.stsci.edu/hlsps/legus/mosaics/ngc5194_ngc5195/"
    "hlsp_legus_hst_acs_ngc5194-ngc5195-mosaic_f814w_v1_sci.fits"
)
image_file = download_file(image_link, cache=True)

catalog_link = (
    "https://jwst-docs.stsci.edu/files/121706162/121706183/1/1625338095703/m51.radec"
)
catalog_file = download_file(catalog_link, cache=True)

Let's read in the file as an HDUList and get a reference position from the associated WCS.

In [ ]:
image_hdul = fits.open(image_file)
image_wcs = WCS(image_hdul[0].header)
ra, dec = image_wcs.wcs.crval
print(f"Starting position: {ra}, {dec}")

NOVT uses the Imviz display tool to show instrument footprints as overlays on FITS images. We can make an Imviz instance directly and load our data into it.

In [ ]:
viz = Imviz()
viz.load_data(image_hdul, data_label="M51")
viz.show()

<a class="anchor" id="visibility"></a>
## Determine visibility ranges

JWST has ranges of time for which a particular target is visible. Within those ranges, the position angle (PA) for an instrument is tightly constrained. We use the NOVT timeline function to derive periods of visibility and the PA for NIRSpec and NIRCam during those times.

Note that this function uses astroquery to find a current ephemeris for JWST, so an internet connection is required to run it.

NOVT interaction tools are classes with displayable widgets in the `widgets` attribute. We can use the timeline widget to create and show a visibility plot for our target position.

In [ ]:
timeline_controls = ShowTimeline()
timeline_controls.ra = ra
timeline_controls.dec = dec
timeline_controls.widgets

Click `Make Timeline Plot` to show the periods of visibility and assigned position angles for NIRSpec and NIRCam over the coming year. Edit the start and end dates to change the displayed date range, or use the buttons at the bottom of the plot to zoom in on a particular time.

Note that as you zoom or pan, the legend will update with the average angle for each instrument in the date range displayed.

<a class="anchor" id="apertures"></a>
## Set aperture locations

We can use NOVT tools to project JWST instrument apertures onto the sky at a given target location and with a specified position angle.

Target location and PA are set separately for NIRSpec and NIRCam, to allow optimization of particular pointings in planning the NIRCam pre-imaging, and to explore a range of possible position angles set by the observation timing.

We use NOVT to make some interactive controls for the NIRSpec and NIRCam apertures.  These are linked to the Imviz instance we created above. The instrument center is automatically populated with the reference RA and Dec from the image we loaded.

In [ ]:
nirspec_controls = ControlInstruments("NIRSpec", viz)
nirspec_controls.widgets

In [ ]:
nircam_controls = ControlInstruments("NIRCam", viz)
nircam_controls.widgets

Add some control buttons to show the instrument overlays. These buttons need to know about the data loaded into the viewer, as well as the state of the instrument controls.

In [ ]:
uploaded_data = UploadData(viz)
uploaded_data.image_file_name = image_file
uploaded_data.has_wcs = True

overlay_controls = ShowOverlays(viz, uploaded_data, nirspec_controls, nircam_controls)
overlay_controls.widgets

Try pressing the control buttons to show the instrument overlays in the Imviz viewer, above.  It may help to pop the viewer into a new window, using the button at the top of the Imviz viewer.

Check the timeline plot for the range of PA values for NIRCam and NIRSpec in the date range you are interested in. Enter these values into the PA widgets in the instrument controls, above.  The displayed footprint in the viewer should change to match. You can also tweak the center position for each instrument or add dithers or mosaic offsets for NIRCam.

<a class="anchor" id="targets"></a>
## Check target coverage

Instrument aperture regions can be used to check whether particular sources are covered by both the NIRSpec MSA apertures and the NIRCam apertures for pre-imaging.

If you have a source catalog in .radec form, the NOVT tools can read it in and create regions identifying the specified sources. This file should have the form of a text file with two or three columns, where the first is RA in degrees, the second is Dec in degrees, and the optional third column flags sources as either primary ('P') or filler ('F').

NOVT tracks catalog files in the uploaded data controls. Let's specify our example catalog.

In [ ]:
uploaded_data.has_catalog = True
uploaded_data.catalog_file = catalog_file

Notice that the buttons above, for primary and filler sources, are now enabled. Try pressing them to show the source locations in the viewer.

<a class="anchor" id="regions"></a>
## Save regions

Regions produced by the NOVT functions can be saved to disk using the astropy regions <a href="https://astropy-regions.readthedocs.io/en/stable/region_io.html">I/O functionality</a>.

We can make a set of regions from the currently configured display.

In [ ]:
save_controls = SaveOverlays(overlay_controls)
all_regions = save_controls.make_regions()

We can save these directly to disk in DS9 format with the write function. This method will also support saving regions in 'fits' or 'crtf' format.

In [ ]:
all_regions.write("novt_overlays.reg", format="ds9", overwrite=True)

However, this function does not currently preserve style information, which makes our regions hard to distinguish in DS9. We can patch it in by serializing the regions to text first, then editing the text to include color and point marker shape.

In [ ]:
colors = {
    "NIRSpec": nirspec_controls.color_primary,
    "NIRCam Short": nircam_controls.color_primary,
    "NIRCam Long": nircam_controls.color_alternate,
    "primary": uploaded_data.color_primary,
    "filler": uploaded_data.color_alternate,
}
markers = {
    "NIRSpec": "cross",
    "NIRCam Short": "cross",
    "NIRCam Long": "cross",
    "primary": "circle",
    "filler": "circle",
}
region_text = all_regions.serialize(format="ds9")
for inst, value in colors.items():
    region_text = region_text.replace(
        f"tag={{{inst}}}", f"tag={{{inst}}} color={value} point={markers[inst]}"
    )
with open("novt_overlays_with_color.reg", "w") as reg_file:
    reg_file.write(region_text)

If you prefer, NOVT has widgets you can use to automatically generate DS9 regions and patch in the style information. Click `Make Region File`, then `Download` to retrieve the file.

Formats other than DS9 are not currently supported.

In [ ]:
save_controls.widgets

[Top of Page](#title)
